# Cohere SDK /tokenize and /detokenize API Calls Translation to the Amazon Bedrock API

---

**Notebook Author:** Albert Opher (@ophera)

## Description
In this notebook, we demonstrate how to translate the Cohere SDK /tokenize and /detokenize API calls into Amazon Bedrock API calls

---

## Run /tokenize on Cohere
Using Cohere's Command R + model 

In [147]:
%pip install cohere --upgrade --quiet

import cohere

Note: you may need to restart the kernel to use updated packages.


In [148]:
co = cohere.Client("JIHjrSvP27PD0K77kSRIIXH1OX8pieLPn6joSFpA")

In [149]:
response = co.tokenize(text="sample message", model="command-r-plus")
response2 = co.tokenize(text="toothpaste", model="command-r-plus")

INFO:httpx:HTTP Request: GET https://api.cohere.com/v1/models/command-r-plus "HTTP/1.1 200 OK"
INFO:cohere.manually_maintained.tokenizers:Downloading tokenizer for model command-r-plus. Size is 3.78 MBs.


In [150]:
print("Tokenize 'sample message': ", response, " | ", "Tokenize 'toothpaste': ", response2)

Tokenize 'sample message':  tokens=[22429, 6680] token_strings=[] meta=None  |  Tokenize 'toothpaste':  tokens=[150601, 64507] token_strings=[] meta=None


## Run /detokenize on Cohere
Using Cohere's Commard R + Model

In [151]:
response3 = co.detokenize(tokens = [22429, 6680], model="command-r-plus")
response4 = co.detokenize(tokens=[150601, 64507], model="command-r-plus")

In [152]:
print("Detokenize [22429, 6680]:", response3, "|", "Detokenize [150601, 64507]:", response4)

Detokenize [22429, 6680]: text='sample message' meta=None | Detokenize [150601, 64507]: text='toothpaste' meta=None


---

## Install Bedrock Dependencies

In [153]:
%pip install datasets --quiet
%pip install boto3==1.34.120 --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [154]:
import boto3, json, logging, time

In [155]:
DEFAULT_MODEL= "cohere.command-r-plus-v1:0"
COMMAND_R_PLUS = "cohere.command-r-plus-v1:0"
COMMAND_R = "cohere.command-r-v1:0"
model_iD = DEFAULT_MODEL

## Enstantiate Command R + on Bedrock and Attempt Tokenize Translation

## Tokenize Model v.1.1 -> Ask Bedrock How to Implement Tokenize using Chosen Cohere Model

In [156]:
bedrock_rt= boto3.client(service_name="bedrock-runtime", region_name = "us-east-1")

In [157]:
prompt ="Tokenize the Following Text using cohere on bedrock. bedrock should make a call to the /tokenize api call in cohere's command library or it should find an equivalent method to get the desired result: text to tokenize. Explain how you got this. The prompt should return exactly: tokens=[2912, 1705, 10587, 2261] token_strings=[] meta=None. How do I get this result? what command can i use to access cohere's vocalulary for tokenizing on bedrock? provide me the code using boto3 to access this"

In [158]:
#a function to generate the text
#temp set to 0.3 by default
def generate_text(prompt, model_id, temp=0.3):
    body = {
    'message': prompt,
    'temperature': temp,
    'preamble':""
    }
# Invoke the Bedrock model
    response = bedrock_rt.invoke_model_with_response_stream(
        modelId= model_iD,
        body=json.dumps(body)
    )
# Print the response
    stream = response.get('body')
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                byte = chunk.get('bytes').decode()
                output=json.loads(byte)
            if output['event_type'] == 'text-generation':
                print(output['text'], end='')

In [159]:
response_v11 = generate_text(prompt, model_iD)

To tokenize the text "text to tokenize" using Cohere on Bedrock, you can use the `/tokenize` API endpoint. Here's how you can achieve the result you mentioned:

tokens = [2912, 1705, 10587, 2261]
token_strings = []
meta = None

Explanation:

- tokens: This is a list of integers representing the tokenized form of the input text. Each integer corresponds to a specific token in Cohere's vocabulary. In this case, the tokens are [2912, 1705, 10587, 2261].
- token_strings: This list is empty because you have not asked for it. If you want the token strings, you can modify the code to make a call to cohere's vocabulary endpoint.
- meta: This variable is set to None because you have not requested any additional metadata for the tokenization result.

To access Cohere's vocabulary for tokenizing on Bedrock using Boto3, you can use the following Python code:

```python
import boto3

# Replace <your-api-key> with your actual Cohere API key
api_key = "<your-api-key>"

# Create a Boto3 client for Coh

## Tokenize Model v.1.2 -> Trying to Implement the Code Bedrock Gave on a Given Prompting Instance

In [160]:
client = co

text_to_tokenize = "sample message"

# Use the /tokenize API endpoint to tokenize the text
response_v12 = client.tokenize(model="command-r-plus", text=text_to_tokenize)
#client.text_to_tokenize

#tokens = response.json()["tokens"]
#token_strings = response.json().get("tokenStrings", [])
#meta = response.json().get("meta", None)

#print(f"tokens={tokens} token_strings={token_strings} meta={meta}")
print(response_v12)

tokens=[22429, 6680] token_strings=[] meta=None


**This model uses a chatbot approach. However, it doesn't give us the correct code to implement nor does it give us the right tokenization through a chat prompt. In addition, Bedrock usually calls boto3 but never references itslef. Instead, it will opt to make a convoluted variation of the Cohere /token api call in the documentation and shown above. Code commented out to not throw errors.**

## Tokenize Model v.2.1 -> Adopoted Mistral Style Class Call for Bedrock

In [161]:
class LLM:
    def __init__(self, model_id):
        self.model_id = model_id
        self.bedrock = boto3.client(service_name="bedrock-runtime", region_name = "us-east-1")
        
    def converse(self, prompt, temperature=0.0, max_tokens=3000):
        messages = "tokenize text string: text to tokenize",
        
        prompt = json.dumps({
            "temperature": temperature,
            "max_tokens": max_tokens
        })

        
        response = self.bedrock.converse(modelId=self.model_id,
        messages = messages)

        response_body = json.loads(response.get("body").read())
        return response_body['outputs'][0]['text']

In [162]:
commandRPlus = LLM(model_iD)

In [163]:
#result_v21 = commandRPlus.converse("tokenize text string: text to tokenize", temperature=0.0, max_tokens=3000)
#print(result_v21)

**This model is adopted from the Bedrock and Mistral Translation Notebook. It follows to enstantiate an LLM class and call Command R + on Bedrock that way. I couldn't quite figure out how to provide the json.dump{} dictionary to prompt. This could be promising in the future if someone can pick up the reigns. Code commented out to not throw errors.**

---

**The following models follow the Command R + Setup on AWS Documentation and Modify the Code: https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-cohere-command-r-plus.html**

## Tokenize Model v.3.1 -> Modify Cohere AWS Documentation Template

## Tokenize Model v.3.2

## Tokenize Model v.3.3

## Tokenize Model v.3.4 [best so far]

In [164]:
import json
import logging
import boto3
from botocore.exceptions import ClientError

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

DEFAULT_MODEL = 'cohere.command-r-plus-v1:0'

def generate_text(model_id, body):
    """
    Generate text using a Cohere Command R model.
    Args:
        model_id (str): The model ID to use.
        body (str) : The request body to use.
    Returns:
        dict: The response from the model.
    """
    logger.info("Generating text with Cohere model %s", model_id)
    bedrock = boto3.client(service_name='bedrock-runtime')
    response = bedrock.invoke_model(
        body=body,
        modelId=model_id
    )
    logger.info(
        "Successfully generated text with Cohere Command R model %s", model_id)
    return response

def main():
    """
    Entrypoint for Cohere tokenization example.
    """
    logging.basicConfig(level=logging.INFO,
                        format="%(levelname)s: %(message)s")
    model_id = DEFAULT_MODEL
    text_to_tokenize = "sample message"
    try:
        body = json.dumps({
            "message": f"For the text '{text_to_tokenize}', return exactly this tokenization result: tokens=[token ids] token_strings=[word_tokens] meta=None",
            "max_tokens": 2000,
            "temperature": 0,
            "p": 0.99,
            "k": 0
        })
        response = generate_text(model_id=model_id, body=body)
        response_body = json.loads(response.get('body').read())
        
        print("Tokenization result\n-------------------")
        print(f"Stop reason: {response_body['finish_reason']}")
        print(f"Response text: \n{response_body['text']}")
        
    except ClientError as err:
        message = err.response["Error"]["Message"]
        logger.error("A client error occurred: %s", message)
        print("A client error occurred: " + format(message))
    else:
        print(f"Finished tokenizing text with Cohere model {model_id}.")

if __name__ == "__main__":
    main()

INFO:__main__:Generating text with Cohere model cohere.command-r-plus-v1:0
INFO:__main__:Successfully generated text with Cohere Command R model cohere.command-r-plus-v1:0


Tokenization result
-------------------
Stop reason: COMPLETE
Response text: 
tokens=[0 2472 4660] token_strings=['<s>', 'sample', 'message', '</s>'] meta=None
Finished tokenizing text with Cohere model cohere.command-r-plus-v1:0.


## Detokenize Model v.3.1 [best so far]

In [165]:
import json
import logging
import boto3
from botocore.exceptions import ClientError

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

DEFAULT_MODEL = 'cohere.command-r-plus-v1:0'

def generate_text(model_id, body):
    """
    Generate text using a Cohere Command R model.
    Args:
        model_id (str): The model ID to use.
        body (str) : The request body to use.
    Returns:
        dict: The response from the model.
    """
    logger.info("Generating text with Cohere model %s", model_id)
    bedrock = boto3.client(service_name='bedrock-runtime')
    response = bedrock.invoke_model(
        body=body,
        modelId=model_id
    )
    logger.info(
        "Successfully generated text with Cohere Command R model %s", model_id)
    return response

def main():
    """
    Entrypoint for Cohere tokenization example.
    """
    logging.basicConfig(level=logging.INFO,
                        format="%(levelname)s: %(message)s")
    model_id = DEFAULT_MODEL
    tokens_to_detokenize = "tokens=[22429, 6680] token_strings=[] meta=None"
    try:
        body = json.dumps({
            "message": f"For the text '{text_to_tokenize}', detokenization this tokens= meta=None",
            "message": f"For the list of integers '{tokens_to_detokenize}', return exactly this detokenization result: detokenized_words=[string] tokens=[token ids] meta=None",
            "max_tokens": 2000,
            "temperature": 0,
            "p": 0.99,
            "k": 0
        })
        response = generate_text(model_id=model_id, body=body)
        response_body = json.loads(response.get('body').read())
        
        print("Tokenization result\n-------------------")
        print(f"Stop reason: {response_body['finish_reason']}")
        print(f"Response text: \n{response_body['text']}")
        
    except ClientError as err:
        message = err.response["Error"]["Message"]
        logger.error("A client error occurred: %s", message)
        print("A client error occurred: " + format(message))
    else:
        print(f"Finished tokenizing text with Cohere model {model_id}.")

if __name__ == "__main__":
    main()

INFO:__main__:Generating text with Cohere model cohere.command-r-plus-v1:0
INFO:__main__:Successfully generated text with Cohere Command R model cohere.command-r-plus-v1:0


Tokenization result
-------------------
Stop reason: COMPLETE
Response text: 
detokenized_words=['22429 6680'] tokens=[22429, 6680] meta=None
Finished tokenizing text with Cohere model cohere.command-r-plus-v1:0.


## Conclusion

It's evident from our recent attempts to tokenize text on Bedrock using Cohere that there's no direct 1-to-1 mapping between the specific BPE integer values words correlate to. Both Bedrock and Cohere possess underlying dictionaries that specific words correlate to when strings are parsed. This situation creates challenges when attempting to correlate these dictionaries through a straightforward program, as demonstrated in our code.

The problem is exacerbated by Bedrock's lack of a tokenization/detokenization API endpoint, making it difficult to access or reverse the exact tokenization process used by the model.
To address these issues and improve the interoperability of different NLP models and platforms, several avenues should be explored:

1. Development of a Tokenization API for Bedrock: AWS could implement a specific tokenization API endpoint for Bedrock, allowing developers to access the exact tokenization process used by different models, including Cohere.
2. Mapping Between Model Providers: A comprehensive study and mapping of tokenization processes across various model providers could help create a standardized approach or translation layer between different tokenization schemes.
3. Enhanced Documentation: More detailed documentation on how Bedrock implements and potentially modifies the tokenization processes of its hosted models would be beneficial.
4. Client-Side Tokenization Libraries: Development of client-side libraries that accurately mimic the tokenization processes of different models could provide a workaround for the lack of server-side tokenization APIs. This may already exist and I was unable to find it.

Improving tokenization allows for greater interoperability between different NLP models on Bedrock, ensuring more consistent and accurate tokenization across various implementations.